In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import np_utils
from keras import applications
from keras.preprocessing import image
from skimage import io, exposure, color, transform
from sklearn.model_selection import train_test_split
import numpy as np
import cPickle as pickle
import h5py as h5py
import os

In [7]:
%matplotlib inline

# constants and hyperparameters
wrk_dir = '/home/ramcharan/deeplearn/caltech/core'
IMG_SIZE = 200
NUM_CLASSES = 101

In [8]:
def preprocess_img(img):
    try:
        hsv = color.rgb2hsv(img)
    except:
        rgb_img = color.gray2rgb(img)
        hsv = color.rgb2hsv(rgb_img)
        
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)
    
    min_side = min(img.shape[:-1])
    center = img.shape[0] // 2, img.shape[1] // 2
    img = img[center[0] - min_side // 2:center[0] + min_side // 2,
              center[1] - min_side // 2:center[1] + min_side // 2,
              :]
    
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))
    
    return img

In [9]:
def get_img_and_classes():
    base_data_dir = '/home/ramcharan/deeplearn/caltech/core/data/categories/'
    
    images = {}
    classes = {}
    
    categories = [categ for categ in os.listdir(base_data_dir)]
    categories.remove('BACKGROUND_Google')
    
    for category in categories:
        print category
        for img_file in os.listdir(base_data_dir + category):
            img = preprocess_img(io.imread(base_data_dir + category + '/' + img_file))
            images[category + '/' + img_file] = img
            classes[category + '/' + img_file] = category
            
    with open('img_dict.p', 'wb') as imd, open('class_dict.p', 'wb') as cld:
        pickle.dump(images, imd, protocol=pickle.HIGHEST_PROTOCOL)
        pickle.dump(classes, cld, protocol=pickle.HIGHEST_PROTOCOL)
        
    return images, classes

In [10]:
if ('img_dict.p' not in os.listdir(wrk_dir)) or ('class_dict.p' not in os.listdir(wrk_dir)):
    images, classes = get_img_and_classes()
else:
    with open('img_dict.p', 'rb') as imd, open('class_dict.p', 'rb') as cld:
        images = pickle.load(imd)
        classes = pickle.load(cld)

In [11]:
x, y = [], []

for img_file in images.keys():
    x.append(images[img_file])
    y.append(classes[img_file])
    
x = np.array(x, dtype='float32')

str_uniq_list, int_coded_uniq_list = np.unique(y, return_inverse=True)
y = np_utils.to_categorical(int_coded_uniq_list, NUM_CLASSES)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [19]:
hehe = applications.VGG19(weights='imagenet', include_top=True)
haha = applications.VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in hehe.layers:
    print layer.name, layer.output_shape
    
# for layer in haha.layers:
#     print layer.name, layer.output_shape

input_15 (None, 224, 224, 3)
block1_conv1 (None, 224, 224, 64)
block1_conv2 (None, 224, 224, 64)
block1_pool (None, 112, 112, 64)
block2_conv1 (None, 112, 112, 128)
block2_conv2 (None, 112, 112, 128)
block2_pool (None, 56, 56, 128)
block3_conv1 (None, 56, 56, 256)
block3_conv2 (None, 56, 56, 256)
block3_conv3 (None, 56, 56, 256)
block3_conv4 (None, 56, 56, 256)
block3_pool (None, 28, 28, 256)
block4_conv1 (None, 28, 28, 512)
block4_conv2 (None, 28, 28, 512)
block4_conv3 (None, 28, 28, 512)
block4_conv4 (None, 28, 28, 512)
block4_pool (None, 14, 14, 512)
block5_conv1 (None, 14, 14, 512)
block5_conv2 (None, 14, 14, 512)
block5_conv3 (None, 14, 14, 512)
block5_conv4 (None, 14, 14, 512)
block5_pool (None, 7, 7, 512)
flatten (None, 25088)
fc1 (None, 4096)
fc2 (None, 4096)
predictions (None, 1000)


In [ ]:
# def setup_for_transfer_learning():
# TRANSFER_LEARNING_SETUP

base_model = applications.VGG19(weights='imagenet', include_top=False, input_shape = (IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False
    
full_model = Sequential()
full_model.add(base_model)
# full_model.add(Dense(512, input_shape=(6, 6, 512), activation='relu'))

full_model.add(Flatten())
full_model.add(Dense(256, activation='relu'))
full_model.add(Dense(256, activation='relu'))
full_model.add(Dropout(0.5))
full_model.add(Dense(NUM_CLASSES, activation='softmax'))

full_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

full_model.fit(x_train, y_train, epochs=20, batch_size=16)

